In [1]:
import pandas as pd
import numpy as np
from scipy.stats import pearsonr

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
train = pd.read_csv('/content/drive/MyDrive/국민대_무역_공모전/데이터/train.csv')
sub = pd.read_csv('/content/drive/MyDrive/국민대_무역_공모전/데이터/sample_submission.csv')

## Step 1: 월별 시계열 데이터 생성

In [4]:
monthly = train.groupby(['item_id', 'year', 'month']).agg({
    'value': 'sum',
    'weight': 'sum',
    'quantity': 'sum',
    'hs4': 'first'
}).reset_index()

monthly['year_month'] = pd.to_datetime(
    monthly['year'].astype(str) + '-' + monthly['month'].astype(str),
    format='%Y-%m'
)

pivot_value = monthly.pivot_table(
    index='year_month',
    columns='item_id',
    values='value',
    fill_value=0
).sort_index()

# Transpose: item을 index로 (예시 코드 형식)
pivot = pivot_value.T  # (100 items × 43 months)

pivot.head(5)

year_month,2022-01-01,2022-02-01,2022-03-01,2022-04-01,2022-05-01,2022-06-01,2022-07-01,2022-08-01,2022-09-01,2022-10-01,...,2024-10-01,2024-11-01,2024-12-01,2025-01-01,2025-02-01,2025-03-01,2025-04-01,2025-05-01,2025-06-01,2025-07-01
item_id,,,,,,,,,,,,,,,,,,,,,
AANGBULD,14276.0,52347.0,53549.0,0.0,26997.0,84489.0,0.0,0.0,0.0,0.0,...,428725.0,144248.0,26507.0,25691.0,25805.0,0.0,38441.0,0.0,441275.0,533478.0
AHMDUILJ,242705.0,120847.0,197317.0,126142.0,71730.0,149138.0,186617.0,169995.0,140547.0,89292.0,...,123085.0,143451.0,78649.0,125098.0,80404.0,157401.0,115509.0,127473.0,89479.0,101317.0
ANWUJOKX,0.0,0.0,0.0,63580.0,81670.0,26424.0,8470.0,0.0,0.0,80475.0,...,0.0,0.0,0.0,27980.0,0.0,0.0,0.0,0.0,0.0,0.0
APQGTRMF,383999.0,512813.0,217064.0,470398.0,539873.0,582317.0,759980.0,216019.0,537693.0,205326.0,...,683581.0,2147.0,0.0,25013.0,77.0,20741.0,2403.0,3543.0,32430.0,40608.0
ATLDMDBO,143097177.0,103568323.0,118403737.0,121873741.0,115024617.0,65716075.0,146216818.0,97552978.0,72341427.0,87454167.0,...,60276050.0,30160198.0,42613728.0,64451013.0,38667429.0,29354408.0,42450439.0,37136720.0,32181798.0,57090235.0


In [ ]:
item_to_hs4 = monthly.groupby('item_id')['hs4'].first().to_dict()

## Step 2: HS4 매핑 및 유사도 함수

In [5]:
# 2-1. 품목별 HS4 매핑
item_to_hs4 = monthly.groupby('item_id')['hs4'].first().to_dict()

def hs4_similarity(item_a, item_b):
    """
    HS4 코드 앞자리 유사도 계산
    예: 8471 vs 8473 -> 2자리 일치 -> 0.5
    """
    str_a = str(item_a).zfill(4)
    str_b = str(item_b).zfill(4)

    if str_a[:2] == str_b[:2]:  # 2자리 일치
        return 0.5
    elif str_a[:1] == str_b[:1]:  # 1자리 일치
        return 0.3
    return 0.0

## Step 3: 통계적 유의성 검증

In [6]:
from scipy.stats import pearsonr

def safe_corr_with_pvalue(x, y):
    if np.std(x) == 0 or np.std(y) == 0:
        return 0.0, 1.0
    corr, p_value = pearsonr(x, y)
    return float(corr), float(p_value) # 피어슨 상관계수, p-value

## Step 4: Granger Causality 검증

- A의 과거 정보가 B의 미래를 예측하는 데 도움이 되는가?"**를 통계적으로 검증하는 방법



```
# B의 과거 + A의 과거 사용
B(3월) = f(B의 1월, B의 2월, A의 1월, A의 2월)
B(4월) = f(B의 2월, B의 3월, A의 2월, A의 3월)
...

예측 오차 = 2  ← 더 정확!
```

**결론**
```
A의 정보를 추가했더니 예측이 더 정확해짐!
→ "A가 B를 Granger-cause 한다" ✅
```

---

🔬 통계적 검증

**가설 검정**
```
H0 (귀무가설): A의 과거 정보는 B 예측에 도움 안 됨
H1 (대립가설): A의 과거 정보는 B 예측에 도움 됨

→ F-test로 검증
→ p-value < 0.05 이면 H0 기각 → A가 B를 cause함!
```



In [ ]:
# from statsmodels.tsa.stattools import grangercausalitytests

# def granger_test(x, y, max_lag=6):
#     """
#     x가 y를 Granger-cause 하는지 검증
#     """
#     try:
#         data = pd.DataFrame({'x': x, 'y': y})
#         test_result = grangercausalitytests(data[['y', 'x']],
#                                             maxlag=max_lag,
#                                             verbose=False)
#         # 각 lag의 p-value 중 최소값 반환
#         p_values = [test_result[lag][0]['ssr_ftest'][1]
#                    for lag in range(1, max_lag+1)]
#         return min(p_values)
#     except:
#         return 1.0


## Step 5: 공행성 강도 점수 계산

In [20]:
def calculate_score_advanced(corr, p_value, lag):
    """
    정교한 공행성 점수 계산 (Granger 제거, 상관계수 중심)
    """

    # 1. 상관계수 점수 (0~75점) - 가장 중요 ⭐⭐⭐
    if abs(corr) >= 0.8:
        corr_score = 75
    elif abs(corr) >= 0.7:
        corr_score = 65
    elif abs(corr) >= 0.6:
        corr_score = 53
    elif abs(corr) >= 0.5:
        corr_score = 40
    else:
        corr_score = abs(corr) * 75  # 0.5 미만은 선형

    # 2. 통계적 유의성 점수 (0~25점) - 신뢰도 확인 ⭐⭐
    if p_value < 0.001:
        sig_score = 25  # 매우 강한 유의성
    elif p_value < 0.01:
        sig_score = 20
    elif p_value < 0.05:
        sig_score = 12
    else:
        sig_score = 5  # 약한 유의성

    # 3. Lag 점수 (0점) - 제거 (이미 상관계수에 반영됨)
    lag_score = 0

    total_score = corr_score + sig_score + lag_score

    return total_score

## Step 6: 각 Leading별 Following 탐지

In [13]:
def grid_search_for_best_f1(pivot):
    """
    F1 Score를 최대화하는 설정 찾기
    """
    results = []

    for corr_th in [0.3, 0.35, 0.4, 0.45, 0.5]:
        for score_th in [25, 30, 35, 40, 45]:
            for min_nz in [8, 10, 12]:
                print(f"테스트: corr={corr_th}, score={score_th}, min_nz={min_nz}")

                pairs = find_comovement_pairs(
                    pivot,
                    corr_threshold=corr_th,
                    score_threshold=score_th,
                    min_nonzero=min_nz
                )

                if len(pairs) == 0:
                    continue

                num_pairs = len(pairs)
                avg_corr = pairs['max_corr'].abs().mean()
                avg_score = pairs['comovement_score'].mean()

                # F1 예상값 계산 (휴리스틱)
                # Recall 근사: 쌍이 많을수록 높음 (정규화)
                recall_proxy = min(1.0, num_pairs / 800)  # 800쌍이면 만점

                # Precision 근사: 상관계수 높을수록 높음
                precision_proxy = avg_corr  # 0.7이면 precision 70%

                # 예상 F1
                if precision_proxy + recall_proxy > 0:
                    estimated_f1 = 2 * precision_proxy * recall_proxy / (precision_proxy + recall_proxy)
                else:
                    estimated_f1 = 0

                results.append({
                    'corr_threshold': corr_th,
                    'score_threshold': score_th,
                    'min_nonzero': min_nz,
                    'num_pairs': num_pairs,
                    'avg_corr': avg_corr,
                    'avg_score': avg_score,
                    'estimated_f1': estimated_f1
                })

                print(f"  → {num_pairs}쌍, 상관={avg_corr:.3f}, 예상F1={estimated_f1:.3f}\n")

    result_df = pd.DataFrame(results)
    result_df = result_df.sort_values('estimated_f1', ascending=False)

    print("="*70)
    print("Top 5 설정 (예상 F1 기준)")
    print("="*70)
    print(result_df.head(10))

    return result_df

# 실행
best_configs = grid_search_for_best_f1(pivot)

테스트: corr=0.3, score=25, min_nz=8


100%|██████████| 100/100 [00:25<00:00,  3.88it/s]


  → 3405쌍, 상관=0.414, 예상F1=0.585

테스트: corr=0.3, score=25, min_nz=10


100%|██████████| 100/100 [00:24<00:00,  4.09it/s]


  → 3319쌍, 상관=0.413, 예상F1=0.585

테스트: corr=0.3, score=25, min_nz=12


...

  → 924쌍, 상관=0.546, 예상F1=0.706

테스트: corr=0.45, score=40, min_nz=12


100%|██████████| 100/100 [00:23<00:00,  4.19it/s]


  → 897쌍, 상관=0.546, 예상F1=0.706

테스트: corr=0.45, score=45, min_nz=8


 22%|██▏       | 22/100 [00:05<00:19,  4.00it/s]


KeyboardInterrupt: 

In [18]:
# from tqdm import tqdm

# def find_comovement_pairs(
#     pivot,
#     max_lag=6,
#     min_nonzero=12,
#     corr_threshold=0.35,
#     score_threshold=30
# ):
#     items = pivot.index.to_list()
#     months = pivot.columns.to_list()
#     n_months = len(months)
#     results = []

#     for i, leader in tqdm(enumerate(items), total=len(items)):
#         x = pivot.loc[leader].values.astype(float)
#         if np.count_nonzero(x) < min_nonzero:
#             continue

#         candidates = []

#         for follower in items:
#             if follower == leader:
#                 continue

#             y = pivot.loc[follower].values.astype(float)
#             if np.count_nonzero(y) < min_nonzero:
#                 continue

#             best_lag = None
#             best_corr = 0.0
#             best_p_value = 1.0

#             for lag in range(1, max_lag + 1):
#                 if n_months <= lag:
#                     continue

#                 corr, p_value = safe_corr_with_pvalue(x[:-lag], y[lag:])

#                 if abs(corr) > abs(best_corr):
#                     best_corr = corr
#                     best_lag = lag
#                     best_p_value = p_value

#             if best_lag is not None and abs(best_corr) >= corr_threshold:
#                 score = calculate_score_advanced(best_corr, best_p_value, best_lag)

#                 # HS4 유사도 계산
#                 hs4_a = item_to_hs4.get(leader)
#                 hs4_b = item_to_hs4.get(follower)

#                 if hs4_a and hs4_b:
#                     str_a = str(hs4_a).zfill(4)
#                     str_b = str(hs4_b).zfill(4)

#                     if str_a[:2] == str_b[:2]:
#                         hs_sim = 0.5
#                     elif str_a[:1] == str_b[:1]:
#                         hs_sim = 0.3
#                     else:
#                         hs_sim = 0.0
#                 else:
#                     hs_sim = 0.0

#                 if score >= score_threshold:
#                     candidates.append({
#                         "following_item_id": follower,
#                         "best_lag": best_lag,
#                         "max_corr": best_corr,
#                         "p_value": best_p_value,
#                         "comovement_score": score,
#                         "hs4_similarity": hs_sim,  # ⭐ 기록
#                     })

#         # 정렬: 1순위 점수, 2순위 HS4 유사도 ⭐
#         candidates.sort(key=lambda x: (x['comovement_score'], x['hs4_similarity']),
#                        reverse=True)

#         for candidate in candidates:
#             results.append({
#                 "leading_item_id": leader,
#                 "following_item_id": candidate["following_item_id"],
#                 "best_lag": candidate["best_lag"],
#                 "max_corr": candidate["max_corr"],
#                 "comovement_score": candidate["comovement_score"],
#             })

#     pairs = pd.DataFrame(results)
#     return pairs

# pairs = find_comovement_pairs(pivot)
# print("탐색된 공행성쌍 수:", len(pairs))
# pairs.head(20)

100%|██████████| 100/100 [00:23<00:00,  4.17it/s]

탐색된 공행성쌍 수: 2178


,leading_item_id,following_item_id,best_lag,max_corr,comovement_score
0,AANGBULD,DEWLVASR,6,0.640221,78.000000
1,AANGBULD,FTSVTTSR,3,0.531400,65.000000
2,AANGBULD,GKQIJYDH,6,0.582501,65.000000
3,AANGBULD,NAQIHUKZ,2,0.525490,65.000000
4,AANGBULD,VUAFAIYJ,2,0.509968,65.000000
5,AANGBULD,ZCELVYQU,5,0.564983,65.000000
6,AANGBULD,ZKENOUDA,1,0.599969,65.000000
7,AANGBULD,LLHREMKS,5,0.499734,57.480043
8,AANGBULD,OKMBFVKS,6,-0.483620,56.271527
9,AANGBULD,APQGTRMF,5,-0.443984,53.298781


In [25]:
from tqdm import tqdm

def find_comovement_pairs(
    pivot,
    max_lag=6,
    min_nonzero=12,
    corr_threshold=0.35,
    score_threshold=30
):
    items = pivot.index.to_list()
    months = pivot.columns.to_list()
    n_months = len(months)
    results = []

    for i, leader in tqdm(enumerate(items), total=len(items)):
        x = pivot.loc[leader].values.astype(float)
        if np.count_nonzero(x) < min_nonzero:
            continue

        candidates = []

        for follower in items:
            if follower == leader:
                continue

            y = pivot.loc[follower].values.astype(float)
            if np.count_nonzero(y) < min_nonzero:
                continue

            best_lag = None
            best_corr = 0.0
            best_p_value = 1.0

            # lag 탐색
            for lag in range(1, max_lag + 1):
                if n_months <= lag:
                    continue

                corr, p_value = safe_corr_with_pvalue(x[:-lag], y[lag:])

                if abs(corr) > abs(best_corr):
                    best_corr = corr
                    best_lag = lag
                    best_p_value = p_value

            # 기본 임계값 통과 시
            if best_lag is not None and abs(best_corr) >= corr_threshold:

                # 종합 점수 계산
                score = calculate_score_advanced(
                    best_corr, best_p_value, best_lag
                )

                # 점수 임계값 통과 시 후보에 추가
                if score >= score_threshold:
                    candidates.append({
                        "following_item_id": follower,
                        "best_lag": best_lag,
                        "max_corr": best_corr,
                        "p_value": best_p_value,
                        "comovement_score": score
                    })

        # 알파벳 순으로 정렬
        candidates.sort(key=lambda x: x['following_item_id'])

        for candidate in candidates:
            results.append({
                "leading_item_id": leader,
                "following_item_id": candidate["following_item_id"],
                "best_lag": candidate["best_lag"],
                "max_corr": candidate["max_corr"]
            })

    pairs = pd.DataFrame(results)
    return pairs

# 실행
pairs = find_comovement_pairs(pivot)
print("탐색된 공행성쌍 수:", len(pairs))
pairs.head(20)

100%|██████████| 100/100 [00:26<00:00,  3.74it/s]

탐색된 공행성쌍 수: 2178


,leading_item_id,following_item_id,best_lag,max_corr
0,AANGBULD,APQGTRMF,5,-0.443984
1,AANGBULD,DDEXPPXU,2,0.383169
2,AANGBULD,DEWLVASR,6,0.640221
3,AANGBULD,DNMPSKTB,4,-0.410635
4,AANGBULD,EVBVXETX,6,0.436623
5,AANGBULD,FTSVTTSR,3,0.531400
6,AANGBULD,GKQIJYDH,6,0.582501
7,AANGBULD,GYHKIVQT,6,-0.405432
8,AANGBULD,KJNSOAHR,3,0.400653
9,AANGBULD,LLHREMKS,5,0.499734


# Step 7. Value 예측

In [16]:
# import os

# # result 폴더 생성 (이미 있으면 무시)
# os.makedirs('./result', exist_ok=True)

# # 파일 저장
# file_name = f'./result/yj_{date_str}.csv'
# submission.to_csv(file_name, index=False)

NameError: name 'date_str' is not defined

In [26]:
import datetime
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from lightgbm import LGBMRegressor
from tqdm import tqdm

train = pd.read_csv('/content/drive/MyDrive/국민대_무역_공모전/데이터/train.csv')

# --- 데이터 전처리 (변경 없음) ---
monthly = (
    train
    .groupby(["item_id", "year", "month"], as_index=False)["value"]
    .sum()
)
monthly["ym"] = pd.to_datetime(
    monthly["year"].astype(str) + "-" + monthly["month"].astype(str).str.zfill(2)
)
pivot = (
    monthly
    .pivot(index="item_id", columns="ym", values="value")
    .fillna(0.0)
)
months_dt = pivot.columns.to_list() # 월(datetime) 리스트

# [중요] 원본 pivot을 사용. S1(F1) 점수를 위해 corr_threshold를 튜닝해야 함.
print("--- '원본 값' 기반 공행성 쌍 탐색 시작 ---")
pairs = find_comovement_pairs(pivot, corr_threshold=0.3)
print("탐색된 공행성쌍 수:", len(pairs))


# --- (2. 학습 데이터 구축 - [수정됨] 피처 엔지니어링) ---
def build_training_data(pivot, pairs, months_dt):
    months = months_dt
    n_months = len(months)
    rows = []

    for row in pairs.itertuples(index=False):
        leader = row.leading_item_id
        follower = row.following_item_id
        lag = int(row.best_lag)
        corr = float(row.max_corr)
        if leader not in pivot.index or follower not in pivot.index:
            continue
        a_series = pivot.loc[leader].values.astype(float)
        b_series = pivot.loc[follower].values.astype(float)

        # [수정] 1년(12개월)치 피처를 사용하기 위해 t의 시작점을 12부터로 변경
        for t in range(max(lag, 12), n_months - 1):
            b_t = b_series[t]
            b_t_1 = b_series[t - 1]
            a_t_lag = a_series[t - lag]
            b_t_plus_1 = b_series[t + 1]

            # [새 피처 1] B의 1년 전 동월 값
            b_t_12 = b_series[t - 12]

            # [새 피처 2] B의 3개월 롤링 평균
            b_roll_mean_3 = (b_series[t] + b_series[t-1] + b_series[t-2]) / 3

            # [새 피처 3] 예측할 달 (t+1)의 월(month) 정보 (계절성)
            target_month = months[t + 1].month

            rows.append({
                "b_t": b_t,
                "b_t_1": b_t_1,
                "a_t_lag": a_t_lag,
                "max_corr": corr,
                "best_lag": float(lag),
                "b_t_12": b_t_12,             # <-- 추가
                "b_roll_mean_3": b_roll_mean_3, # <-- 추가
                "month": float(target_month), # <-- 추가
                "target": b_t_plus_1,
            })
    df_train = pd.DataFrame(rows)
    return df_train

df_train_model = build_training_data(pivot, pairs, months_dt)
print('생성된 학습 데이터의 shape :', df_train_model.shape)


# --- (3. 회귀모델 학습 - [수정됨] 피처 리스트) ---
# [수정] 5개 -> 8개 피처로 변경
feature_cols = [
    'b_t', 'b_t_1', 'a_t_lag', 'max_corr', 'best_lag',
    'b_t_12', 'b_roll_mean_3', 'month' # <-- 추가
]

if df_train_model.empty:
    print("오류: 학습 데이터가 없습니다. (lag가 너무 크거나, t 시작점이 높을 수 있음)")
    submission = pd.DataFrame(columns=['leading_item_id', 'following_item_id', 'value'])
else:
    train_X = df_train_model[feature_cols].values
    train_y = df_train_model["target"].values

    reg = LGBMRegressor(random_state=42, n_estimators=500, learning_rate= 0.01)
    print("LGBMRegressor 모델(피처 추가) 학습 시작...")
    reg.fit(train_X, train_y)
    print("모델 학습 완료.")


# --- (4. 예측 - [수정됨] 피처 생성) ---
def predict(pivot, pairs, reg, months_dt):
    months = months_dt
    n_months = len(months)
    t_last = n_months - 1
    t_prev = n_months - 2
    preds = []

    # 예측할 달(2025년 8월)의 '월' 정보
    target_month = months[-1].month + 1 if months[-1].month < 12 else 1

    for row in tqdm(pairs.itertuples(index=False)):
        leader = row.leading_item_id
        follower = row.following_item_id
        lag = int(row.best_lag)
        corr = float(row.max_corr)
        if leader not in pivot.index or follower not in pivot.index:
            continue
        a_series = pivot.loc[leader].values.astype(float)
        b_series = pivot.loc[follower].values.astype(float)

        # [수정] 12개월치 피처를 쓰기 위해 t_last가 12보다 큰지 확인
        if t_last - lag < 0 or t_last < 12:
            continue

        b_t = b_series[t_last]
        b_t_1 = b_series[t_prev]
        a_t_lag = a_series[t_last - lag]

        # [새 피처]
        b_t_12 = b_series[t_last - 12]
        b_roll_mean_3 = (b_series[t_last] + b_series[t_last-1] + b_series[t_last-2]) / 3

        # [수정] 8개 피처로 X_test 구성
        X_test = np.array([[
            b_t, b_t_1, a_t_lag, corr, float(lag),
            b_t_12, b_roll_mean_3, float(target_month) # <-- 추가
        ]])

        y_pred = reg.predict(X_test)[0]

        y_pred = max(0.0, float(y_pred))
        y_pred = int(round(y_pred))

        preds.append({
            "leading_item_id": leader,
            "following_item_id": follower,
            "value": y_pred,
        })
    df_pred = pd.DataFrame(preds)
    return df_pred

if df_train_model.empty:
    print("학습된 모델이 없어 예측을 건너뜁니다.")
else:
    print("--- '피처 추가' 모델 예측 시작 ---")
    submission = predict(pivot, pairs, reg, months_dt)
    print(submission.head())

date_str = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
file_name = f'./result/sy_{date_str}.csv'
submission.to_csv(file_name, index=False)


print(f"✓ 파일 저장 완료: {file_name}")
print(f"LGBM(원본, 피처추가) 모델 예측 완료. {file_name}.csv 저장됨.")

--- '원본 값' 기반 공행성 쌍 탐색 시작 ---


100%|██████████| 100/100 [00:22<00:00,  4.35it/s]


탐색된 공행성쌍 수: 2909
생성된 학습 데이터의 shape : (87270, 9)
LGBMRegressor 모델(피처 추가) 학습 시작...
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002295 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1550
[LightGBM] [Info] Number of data points in the train set: 87270, number of used features: 8
[LightGBM] [Info] Start training from score 4074946.938707
모델 학습 완료.
--- '피처 추가' 모델 예측 시작 ---


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/usr/local/lib/pyth

  leading_item_id following_item_id    value
0        AANGBULD          APQGTRMF   131655
1        AANGBULD          BEZYMBBT  3849252
2        AANGBULD          DDEXPPXU   131655
3        AANGBULD          DEWLVASR   339399
4        AANGBULD          DNMPSKTB  5284960
✓ 파일 저장 완료: ./result/sy_20251117_093248.csv
LGBM(원본, 피처추가) 모델 예측 완료. ./result/sy_20251117_093248.csv.csv 저장됨.
